##vLLM과 파인 튜닝 모델의 RAG성능 평가
RAG(Retrieval‑Augmented Generation) 모델의 성능을 평가하기 위해서는 단순한 텍스트 생성 품질뿐만 아니라 검색된 문서의 관련성과 활용도를 종합적으로 분석해야 합니다. 이번 단계에서는 학습된 RAG 모델을 vLLM 으로 로드합니다.
학습된 모델의 성능을 평가하기 위해서는 효율적인 추론 환경이 필요합니다. vLLM 은 대규모 언어 모델과 멀티모달 모델의 추론 속도를 향상시키는 추론 엔진입니다. 이번 단계에서는 vLLM 을 사용하여 업로
드된 모델을 로드하고, 테스트 데이터에 대한 추론을 수행한 후 LLM 기반 평가와 정량적 평가를 진행해보겠습니다.


###1. 패키지 임포트 및 vLLM객체 설정

In [ ]:
import pandas as pd
import json
import re
from typing import List
from openai import OpenAI
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import load_from_disk
from vllm import LLM, SamplingParams

필요한 라이브러리들을 import 합니다. pandas는 데이터 처리와 결과 분석을 위해, json은 JSON
파 싱 을 위 해, re는 정 규 표 현 식 처 리 를 위 해 사 용 합 니 다. typing.List는 타 입 힌 트 를 위 해,
openai.OpenAI는 GPT 모델을 활용한 평가를 위해, tqdm은 진행 상황 표시를 위해 사용합니다.
transformers.AutoTokenizer는 토크나이저 로드를 위해, datasets.load_from_disk는
저장된 데이터셋 로드를 위해, vllm.LLM과 SamplingParams는 고성능 추론을 위해 사용합니다.

In [ ]:
vllm_model = LLM(
    model="iamjoon/qwen2-7b-rag-ko-checkpoint-285",
    dtype="bfloat16",
)

vLLM 을 사용하여 학습된 RAG 모델을 로드합니다. model 매개변수에는 허깅페이스 허브에 업로드된
모델 경로를 지정합니다. dtype="bfloat16"으로 설정하여 메모리 효율성을 높이면서도 모델 성능
을 유지합니다. bfloat16 은 GPU 메모리 사용량을 절반으로 줄이면서도 float32 와 유사한 수치 안정성을
제공합니다.

###2. 테스트 데이터 로드 전처리
저장된 테스트 데이터셋을 로드하고 평가를 위해 데이터를 전처리합니다.

In [ ]:
test_dataset = load_from_disk('test_dataset')
tokenizer = AutoTokenizer.from_pretrained("iamjoon/qwen2-7b-rag-ko-checkpoint-285")

prompt_lst = []
label_lst = []
questions = []
contexts = []
for prompt in test_dataset["messages"]:
  text = tokenizer.apply_chat_template(
    prompt, tokenize=False, add_generation_prompt=False)
  input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
  label = text.split('<|im_start|>assistant')[1].split('<|im_end|>')[0]
  question = text.split('<|im_start|>user')[1].split('<|im_end|>')[0].strip()
  context = text.split('검색 결과:\n-----')[1].split('<|im_end|>')[0].strip()
  prompt_lst.append(input)
  label_lst.append(label)
  questions.append(question)
  contexts.append(context)


load_from_disk()를 사용하여 이전에 저장한 테스트 데이터셋을 불러옵니다. 동일한 모델의 토크나이저를 로드하여 일관된 텍스트 처리를 보장합니다.

각 프롬프트에 대해 apply_chat_template()를 호출하여 테스트 데이터를 챗 템플릿으로 변환합니다. 그 후 변환된 텍스트에서 입력 부분과 정답 레이블을 분리합니다. '<|im_start|>assistant
'를 기준으로 분할하여 모델에게 제공할 입력 부분만 추출하여 input에 저장합니다. 정답 레이블은 assistant 응답에서 '<|im_end|>' 토큰을 제거하여 순수한 텍스트만 추출하여 label에 저장합니다.

질문과 검색 결과 또한 별도로 추출합니다. 사용자 메시지에서 질문 부분을 추출하고, '검색 결과:' 다음에 오는 검색 결과를 분리합니다. 이는 뒤에서 RAG 를 평가할 때 질문‑검색 결과‑답변 간의 관계를 분석
하기 위함입니다. 10 번 질문과 레이블을 출력하여 데이터 처리가 올바르게 수행되었는지 검증합니다

In [ ]:
questions[10]

In [ ]:
label_list[10]

해당 질문에 대한 정답 레이블입니다.

###3. 모델 추론

In [ ]:
# 생성 파라미터 설정
sampling_params = SamplingParams(
temperature=0,
max_tokens=1024,
)

모델 추론을 위한 샘플링 매개변수를 설정합니다. temperature=0으로 설정하여 랜덤성을 주지 않고 결정적인 출력을 얻도록 합니다.이는 일관된 평가를 위해 중요합니다. max_tokens=1024로 설정하여 충분히 긴 응답을 생성할 수 있도록 합니다.

이후 상위 50 개 샘플에 대해 배치 추론을 실행합니다. 전체 테스트 데이터에 대해서도 수행하여도 되지만 이번 실습에서는 50 개 정도만 평가해보겠습니다.


In [ ]:
preds = vllm_model.generate(prompt_lst[:50], sampling_params)

vLLM 의 배치 처리 기능을 활용하여 빠르게 추론을 수행합니다. 이제 평가를 위해서 생성된 결과에서 텍스트만 추출합니다.

In [ ]:
preds = [pred.outputs[0].text for pred in preds]

### 4. LLM기반의 평가
RAG답변을 GPT-4로 평가해 보겠습니다. OpenAI클라이언트를 초기화합니다. GPT-4를 활용한 정성적 평가를 수행하기 위해 API키를 설정합니다.


In [ ]:
# OpenAI API 초기화
client = OpenAI(api_key="여 러 분 의 키 값") # 본인의 API키로 교체 하세요

키 값을 설정했스니 이제 GPT-4를 이용한 평가 함수를 작성합니다.

In [ ]:
def evaluate_rag_with_llm(questions, contexts, predictions, labels):
  """
  LLM 평가 메트릭으로 RAG 시스템 평가
  Args:
  questions: 질문 리스트
  contexts: 검색 결과 리스트
  predictions: 예측 리스트
  labels: 레이블 리스트
  Returns:
  평가 결과가 포함된 데이터 프레임
  """

  # 결과를 저장할 리스트
  results = []

  # 평가 프롬프트
  prompt_template = """
당신은 RAG(Retrieval-Augmented Generation) 시스템 평가 전문가입니다. 아래 정보를
  바탕으로 생성된 답변의 품질을 철저히 평가해주세요

질문: {question}

검색된 컨텍스트:
{context}

생성된 답변:
{prediction}

참조 답변(정답):
{label}

다음 4가지 평가 기준으로 1-5점 척도로 점수를 매겨주세요:

1. 응답 정확성 (Answer Correctness) [1-5]:
  * 생성된 답변이 참조 답변과 비교하여 정확하고 완전한 정보를 제공하는지 평가
  * 1점: 완전히 잘못된 정보
  * 2점: 부분적으로 관련된 정보를 담고 있으나 대부분 부정확함
  * 3점: 정확한 정보와 부정확한 정보가 혼재되어 있음
  * 4점: 대부분 정확하지만 일부 정보가 누락되거나 미미한 오류가 있음
  * 5점: 참조 답변과 비교했을 때 완전히 정확하고 포괄적인 정보를 제공함

2. 컨텍스트 관련성 (Context Relevance) [1-5]:
  * 검색된 컨텍스트가 질문에 대답하기 위해 관련성이 높은지 평가
  * 1점: 컨텍스트가 질문과 전혀 관련이 없음
  * 2점: 컨텍스트가 질문과 간접적으로만 관련됨
  * 3점: 컨텍스트 중 일부만 질문과 직접적으로 관련됨
  * 4점: 대부분의 컨텍스트가 질문과 직접적으로 관련됨
  * 5점: 모든 컨텍스트가 질문에 완전하게 관련되어 있고 불필요한 정보가 없음

3. 컨텍스트 충실성 (Context Faithfulness) [1-5]:
  * 생성된 답변이 주어진 컨텍스트에만 기반하는지, 아니면 없는 정보를 추가했는지 평가
  * 1점: 답변이 컨텍스트에 없는 정보로만 구성됨 (심각한 환각)
  * 2점: 답변이 주로 컨텍스트에 없는 정보로 구성됨
  * 3점: 답변이 컨텍스트 정보와 없는 정보가 혼합되어 있음
  * 4점: 답변이 주로 컨텍스트에 기반하지만 약간의 추가 정보가 있음
  * 5점: 답변이 전적으로 컨텍스트에 있는 정보만을 사용함

4. 컨텍스트 충분성 (Context Recall)[1-5]:
  * 검색된 컨텍스트가 질문에 완전히 답변하기에 충분한 정보를 포함하는지 평가
  * 1점: 컨텍스트가 답변에 필요한 정보를 전혀 포함하지 않음
  * 2점: 컨텍스트가 필요한 정보의 일부만 포함함
  * 3점: 컨텍스트가 필요한 정보의 약 절반만 포함함
  * 4점: 컨텍스트가 필요한 정보의 대부분을 포함하지만 일부 누락됨
  * 5점: 컨텍스트가 질문에 완전히 답변하기 위한 모든 필요한 정보를 포함함

반드시 다음 JSON 형식으로만 응답하세요. 마크다운은 사용하지 않습니다.:
{
  "answer_correctness": 정수로 된 점수(1-5),
  "context_relevance": 정수로 된 점수(1-5),
  "context_faithfulness": 정수로 된 점수(1-5),
  "context_recall": 점수(1-5),
  "analysis": "종합적인 분석 의견"
}

다른 형식의 응답은 하지 마세요. 오직 마크다운이 아닌 JSON만 변환하세요.
"""
  # 각 항목에 대해 평가 수행
  for i in tqdm(range(len(questions)), total=len(questions), desc="RAG 평가 진행중"):
    try:
      # 프롬프트 생성 - format 대신 replace 사용
      prompt = prompt_template
      prompt = prompt.replace("{question}", str(questions[i]) if questions[i] is not None else "")
      prompt = prompt.replace("{context}", str(contexts[i]) if contexts[i]is not None else "")
      prompt = prompt.replace("{prediction}", str(predictions[i]) ifpredictions[i] is not None else "")
      prompt = prompt.replace("{label}", str(labels[i]) if labels[i] is notNone else "")


      # GPT-4 API 호출
      response = client.chat.completions.create(
          model="gpt-4o",
          messages=[
              {"role": "system", "content": "당신은 RAG 평가 도구 입니다. 반드시 유효한 JSON형식으로만 응답하세요"},
              {"role": "user", "content": prompt}
          ],
          temperature=0,
          response_format={"type": "json_object"}
      )

      # 결과 파싱
      result = json.loads(response.choices[0].message.content)

      # 개별 메트릭 점수 추출
      answer_correctness = result['answer_correctness']
      context_relevance = result['context_relevance']
      context_faithfulness = result['context_faithfulness']
      context_recall = result['context_recall']

      # 총점 직접 계산 (개별 메트릭의 합)
      total_score = answer_correctness + context_relevance + context_faithfulness + context_recall

      # 원본 데이터와 평가 결과 결합
      row_result = {
          'id': i,
          'question': questions[i],
          'answer_correctness': answer_correctness,
          'context_relevance': context_relevance,
          'context_faithfulness': context_faithfulness,
          'context_recall': context_recall,
          'total_score': total_score,
          'analysis': result['analysis']
      }

      results.append(row_result)

      except Exception as e:
        print(f"항목 {i}평가 중 오류 발생: {e}")
        results.append({
            'id': i,
            'question': questions[i],
            'error': str(e)
        })

# 결과 데이터 프레임 생성
result_df = pd.DataFrame(results)

# 요약 통계 계산
if 'total_score' in results_df.columns:
  metrics_summary = {
      '평균 총점': results_df['total_score'].mean(),
      '응답 정확성 평균': results_df['answer_correctness'].mean(),
      '컨텍스트 관련성 평균': results_df['context_relevance'].mean(),
      '컨텍스트 충실성 평균': results_df['context_faithfulness'].mean(),
      '컨텍스트 충분성 평균': results_df['context_recall'].mean()
}
print("\n===== 평 가 요 약 =====")
for metric, value in metrics_summary.items():
  print(f"{metric}: {value:.2f}")

return results_df, metrics_summary if 'total_score' in results_df.columns else results_df




RAG 시스템을 종합적으로 평가하는 함수를 정의하였습니다. 이 함수는 GPT‑4 를 활용하여 4 가지 핵심 메트릭으로 평가를 수행합니다. 코드를 살펴봅시다.
함수 내부에서 결과를 저장할 빈 리스트를 초기화합니다. prompt_template에는 GPT‑4 에게 전달할 상세한 평가 기준과 지침이 포함되어 있습니다. 각 평가 기준은 1‑5 점 척도로 명확하게 정의되어 있습니다.

각 질문에 대해 반복문을 실행하며 tqdm 을 사용하여 진행 상황을 표시합니다. GPT‑4 API 를 호출할 때 model="gpt-4o"를 사용하여 GPT‑4o 모델을 활용하고, temperature=0으로 설정하여 일관된 평가를 보장합니다. response_format={"type": "json_object"}를 설정하여 응답이 JSON 형식으로 반환되도록 강제합니다. 이후 API 응답을 JSON 으로 파싱하고 각 메트릭 점수를 추출하여 4 개 메트릭의 합을 총점으로 계산하여 종합적인 성능 지표를 제공합니다. 평가 중 오류가 발생하면 오류 메시지를 출력하고 해당 정보를 결과에 기록합니다.


이제 50 개 샘플에 대해 LLM 기반 평가를 실행합니다. 모든 평가가 완료되면 pandas DataFrame 으로 결과를 구성하고, 평균 점수를 계산하여 요약 통계를 제공합니다.

In [ ]:
results_df, metrics_summary = evaluate_rag_with_llm(questions[:50], contexts[:50], preds[:50], label_lst[:50])

평가가 완료되었습니다. 50 개 샘플을 2 분 34 초 동안 처리하여 샘플당 약 3.1 초가 소요되었습니다. 평가 결과를 보면 전반적으로 준수한 성능을 보이고 있습니다. 특히 컨텍스트 충분성 (4.94) 과 컨텍스트 관련성 (4.88) 이 높게 나타났으며, 이는 검색 시스템이 질문과 관련된 적절한 문서를 잘 찾아내고 있음을 의미
합니다. 응답 정확성 (4.30) 과 컨텍스트 충실성 (4.52) 도 양호한 수준입니다. 평가 결과 상위 5 개 샘플을 확인합니다.

In [ ]:
results_df.head()

첫 번째 샘플 (id=0) 은 총점 12 점으로 낮은 성능을 보였는데, 이는 응답 정확성과 컨텍스트 충실성에서 1점을 받았기 때문입니다. 반면 2‑4 번 샘플들은 모든 메트릭에서 5 점을 받아 총점 20 점의 준수한 성능을 보였습니다.

###5. 정량 평가
RAG 시스템의 성능을 정량적으로 측정하기 위해서는 생성된 답변이 올바른 문서를 참조했는지 확인해야 합니다. 우리가 파인튜닝한 모델은 답변을 생성할 때 참조한 문서 번호를 [[ref1]], [[ref2]] 형태로 명시하도록 훈련되었습니다. 이러한 참조 표기는 모델이 어떤 문서를 근거로 답변을 생성했는지 추적할 수 있게 해줍니다.

정답 레이블에도 동일한 형태로 참조 번호가 표기되어 있으므로, 모델의 예측과 정답을 비교하여
참조 정확성을 측정할 수 있습니다. 예를 들어 정답에서 문서 1 번과 3 번을 참조했다면 [[ref1]],
[[ref3]]이 포함되어 있을 것이고, 모델도 같은 문서들을 참조했는지 확인할 수 있습니다. 이를 통해 모델이 관련성 높은 문서를 식별하고 활용하는 능력을 평가할 수 있습니다.

In [ ]:
def extract_ref_numbers(text: str) --> List[int]:
  """
  텍스트에서 [[ref숫자]] 패턴의 숫자들을 추출하여 중복 없이 정렬된 리스트로 반환 합니다.

  Args:
    text (str): 분석할 텍스트

  Returns:
    List[int]: 추출된 숫자들의 정렬된 리스트

  Example:
    >>> text = "이것은 [[ref1]]과 [[ref2]], [[ref1]]입니다."
    >>> extract_ref_numbers(text)
    [1, 2]
  """
  # [[ref숫자]] 패턴을 찾는 정규 표현식
  pattern = r'\[\[ref(\d+)\]\]'

  # 모든 매치를 찾아서 숫자만 추출
  numbers = [int(match.group(1)) for match in re.finditer(pattern, text)]

  # 중복 제거하고 정렬하여 반환
  return sorted(list(set(numbers)))

함수의 동작을 확인하기 위해 테스트 데이터의 레이블 중 1 개를 임의 선정하여 함수의 결과를 확인해보겠습니다.

In [ ]:
label = '''제3차 중동 전쟁과 아랍의 겨울의 주요 원인과 결과는 다음과 같습니다.

### 제3차 중동 전쟁 (6일 전쟁)
**주요 원인**
1. **티란 해협 봉쇄:** 이집트가 이스라엘 선박의 티란 해협 통과를 봉쇄하면서 긴장이
 고조되었습니다. 이스라엘은 이를 명분으로 삼았습니다[[ref1]].
2. **군사적 준비:** 이집트는 이스라엘과의 국경에 군을 배치하고, 이스라엘은 이에 대
응해 예방적 공습을 감행했습니다[[ref1]].

**주요 결과**
1. **영토 확장:** 이스라엘은 가자 지구, 시나이 반도, 동예루살렘, 요르단 강 서안 지
구, 골란 고원을 점령하여 영토를 3배로 확장했습니다[[ref1]].
2. **난민 발생:** 전쟁으로 인해 약 100,000명의 시리아인과 300,000명의 팔레스타인인
이 난민이 되었습니다[[ref1]].
3. ** 군사저 자만:** 이스라엘의 승리는 군사적 자만을 초래하여 이후 1973년 욤키푸르
전쟁에서 아랍 연합군의 초기 승리를 가능하게 했습니다[[ref1]].

### 아랍의 겨울
**주요 원인:**
1. **아람의 봄의 실패:** 2011년 아랍의 봄 이후 민주호와 개혁을 목표로 한 시도들이
실패하면서 권위주의가 복고되고, 사회적 불안정이 증가했습니다[[ref4]].
2. **이슬람 극단주의의 부상:** 이슬람 극단주의와 테러 단체들이 활동을 강화하면서
지역 내 폭력과 혼란이 가중되었습니다[[ref4]].

**주요 결과:**
1. **내전과 사회 불안정:** 시리아 내전, 이라크 내전, 예맨 내전 등 여러 지역에서 내
전과 사회 불안정이 발생했습니다[[ref4]].
2. **경제적 손실:** 아랍의 겨울로 인해 발생한 사회적 손실비용은 약 8천억 달러에 달
하며, 수백만 명의 난민이 발생했습니다[[ref4]].
3. **인도적 위기:** 시리아, 이집트, 이라크 등 여러 지역에서 인도적 지원이 필요한
사람들의 수가 급증했습니다[[ref4]].

이와 같이 제3차 중동 전쟁과 아랍의 겨울은 각각 중동 지역의 군사적, 정치적, 사회적
변화를 초래한 중요한 사건들입니다.'''

cited_docs = extract_ref_numbers(label)
print(cited_docs)


함수가 작동하여 텍스트에서 참조 번호 1 과 4 를 추출했습니다. 이제 테스트 데이터에 대해서 모델의 예측을 가정하여 이 또한 함수의 결과를 얻어보겠습니다

In [ ]:
model_prediction = '''제3차 중동 전쟁과 아랍의 겨울은 각각 다른 시기에 발생한 사건
이며, 주요 원인과 결과는 다음과 같습니다.

제3차 중동 전쟁은 1967년 6월에 시작된 이스라엘과 주변 국가인 이집트, 요르단, 시리
아, 레바논 사이의 전쟁입니다. 이 전쟁의 주요 원인은 이스라엘과 주변 국가 사이
의 긴장과 불안정한 관계였습니다. 이스라엘은 1956년 수에즈 위기 때 주요 목표 중
 하나였던 티란 해협을 확보하려고 했고, 이 지역은 1950년 이래 이집트가 이스라엘
 의 항해를 봉쇄하고 있었습니다. 이스라엘은 이 지역의 봉쇄를 계속해서 유지하려고
 했지만, 이는 결국 1967년 6월에 이스라엘과 주변 국가 사이의 전쟁으로 이어졌습
 니다.

 전쟁의 결과로, 이스라엘은 이집트로부터 가자 지구와 시나이 반도를, 요르간으로부터
  동 예루살렘과 요르단 강 서안 지구를, 시리아로부터 골란 고원을 획득했습니다. 이
  스라엘의 영토는 3배나 커졌고, 이스라엘군은 군사적 승리를 거두었습니다. 그러나
  이 전쟁으로 인해 군사들은 자만감에 빠졌고, 이는 1973년 요키푸르 전쟁 당시 아랍
  연합군이 초기에 승리를 거둔는 계기가 되었습니다. 또한, 이 전쟁으로 인해
  100,000명의 시리아인이 골란 고원을 떠났고, 300,000명의 팔레스타인인이 서안 지
  구를 떠났는데 이들은 모두 난민이 되었습니다.

  아랍의 겨울은 2011년 아랍의 봄 이후 아랍 일대에서 벌어지고 있는 광범위한 혼란과 폭
  력과 사회불안정 사태를 가리키는 말입니다. 이 겨울에는 시리아 내전, 이라크 내전
  , 이집트 위기, 예멘 내전이 모두 포함됩니다. 권위주의가 복고되고 자유 민권이 억
  압되고 있으며, 이슬람 극단주의와 테러 단체가 곳곳에서 준동하고 있습니다. 아랍
  의 겨울은 복수의 지역에서 내란과 내전이 발생하고, 사회가 불안정해지며, 아랍 지
  역의 경제와 인구가 쇠퇴하고 인종주의와 종교적 종파주의가 판을 치는 것으로 특징
  지어집니다. 아랍의 겨울의 가장 극단적 사례는 시리아입니다. 바샤르 알 아사드 대
  통령의 독재에 반대하여 일어난 시위에 알누스라 전선 등의 이슬람주의 집단이 개입
  하고, 자유 시리아군의 부패와 범죄가 밝혀지면서 이슬람주의자들의 입지가 강화되
  었습니다. 그리하여 시리아는 독재정부와 반정부 반군과 이슬람주의 테러리스트들의
  삼파내전에 휩싸이게 되었으며, 그 여파는 이웃나라인 레바논과 이라크까지 번지고
  있습니다. [[ref1]]'''

model_pred = extract_ref_numbers(model_prediction)
print(model_pred)

모델 예측에서는 ref1 만 참조했음을 확인할 수 있습니다. 정답에서는 [1, 4] 를 참조했으나 모델은 [1] 만 참조하여 일부 문서를 놓쳤음을 알 수 있습니다. 이렇게 레이블과 모델의 예측의 차이를 가지고 정량적으로 평가하는 것이 가능할 것입니다. 이에 대한 정확성을 측정하기 위한 F1 스코어 계산 함수를 정의합니다. F1 스코어는 하나의 데이터에 다수의 정답이 달려있는 사정을 가정하는 다중 레이블 분류 문제에 최적화되어 있어, 여러 문서를 참조하는 RAG 시스템의 평가에 적합합니다. 위의 예시만 해도 하나의 테스트
데이터에 [1, 4] 로 두 개의 문서 번호. 즉, 두 개의 레이블이 존재하기 때문입니다.


In [ ]:
def calculate_f1_score(labels, predictions):
  """
  F1 score 계산 (multilabel 케이스) - 이중 리스트 입력 지원

  Args:
    labels: 샘플별 실제 레이블이 담긴 리스트 또는 이중 리스트
    predictions: 샘플별 예측 레이블이 담긴 리스트 또는 이중 리스트

  Returns:
    F1 점수
  """
  # 입력 형식 확인
  is_nested_list = isinstance(labels, list) and len(labels) > 0 and isinstance(labels[0], list)

  # 이중 리스트가 아니면 이중 리스트로 변환
  if not is_nested_list:
    labels = [labels]
    predictions = [predictions]

  # 길이 확인
  if len(labels) != len(predictions):
    raise ValueError("라 벨 과 예 측 의 길 이 가 일 치 하 지 않 습 니 다.")

  # 전체 통계 초기화
  total_true_positives = 0
  total_false_positives = 0
  total_false_negatives = 0

  # 각 샘플에 대해 계산
  for sample_labels, sample_preds in zip(labels, predictions):
    # 빈 리스트는 빈 집합으로 변환
    if not sample_labels:
      sample_labels = set()
    else:
      sample_labels = set(sample_labels)
    if not sample_preds:
      sample_preds = set()
    else:
      sample_preds = set(sample_preds)

    # 통계 계산
    true_positives = len(sample_labels.intersection(sample_preds))
    false_positives = len(sample_preds - sample_labels)
    false_negatives = len(sample_labels - sample_preds)

    # 전체 통계에 추가
    total_true_positives += true_positives
    total_false_positives += false_positives
    total_false_negatives += false_negatives

  # 특수 케이스: 둘다 빈 리스트인 경우는 F1 = 1.0으로 처리
  if sum(bool(l) for l in labels) == 0 and sum(bool(p) for p in predictions) ==0:
    return 1.0

  # 정밀도 와 재현율 계산
  precision = total_true_positives / (total_true_positives + total_false_positives) if (total_true_positives + total_false_positives) > 0 else 0
  recall = total_true_positives / (total_true_positives + total_false_negatives) if (total_true_positives + total_false_negatives) > 0 else 0
  # F1 점수 계산
  f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
  return f1


하나의 답변은 여러 문서를 동시에 참조할 수 있으므로 다중 레이블 분류로 접근합니다. 모든 데이터의 통계를 누적한 후 전체 정밀도와 재현율을 계산합니다. 특별한 경우로 정답과 예측이 모두 빈 리스트인 경우는 F1=1.0 으로 처리합니다. 이는 “참조할 문서가 없는 상황을 올바르게 인식했다” 고 판단하기 때문입니다. 함수의 동작을 확인하기 위해 5 가지 시나리오로 테스트해봅니다

In [ ]:
test_cases = [
    ([1, 4], [1]), # 기본 케이스
    ([1, 2, 3], [1, 2, 3]), # 완벽한 예측
    ([1, 2, 3], [4, 5, 6]), # 완전히 틀린 예측
    ([1], [1, 2, 3]), # 과대 예측
    ([], []), # 빈 리스트
]
for labels, predictions in test_cases:
  f1 = calculate_f1_score([labels], [predictions])
  print(f"Labels: {labels}")
  print(f"Predictions: {predictions}")
  print(f"F1 Score: {f1:.4f}")
  print("-" * 30)


테스트 결과를 분석하면 함수가 올바르게 작동함을 확인할 수 있습니다. 첫 번째 케이스 ([1,4] vs [1]) 에서 F1 스코어가 0.6667 인 것은 정밀도 1.0(예측한 것은 모두 맞음), 재현율 0.5(정답 중 절반만 맞춤) 의 조화평균입니다. 완벽한 예측은 1.0, 완전히 틀린 예측은 0.0, 과대 예측은 0.5, 빈 리스트는 1.0 의 결과를 보임을 확인하였습니다.
이제 실제 테스트 데이터 상위 50 건의 레이블과 모델의 예측에 대해서 평가를 진행해보겠습니다. 먼저 50 개 샘플에 대해 정답 레이블과 모델 예측에서 참조 번호를 추출합니다.

In [ ]:
label_ref_numbers = []
pred_ref_numbers = []

for label, pred in zip(label_lst[:50], preds[:50]):
  label_ref_numbers.append(extract_ref_numbers(label))
  pred_ref_numbers.append(extract_ref_numbers(pred))

각 샘플에 대해 extract_ref_numbers() 함수를 적용하여 참조된 문서 번호 리스트를 생성하였습
니다. 50 개의 테스트 데이터 중 임의로 20‑29 번 샘플의 정답 참조 번호를 확인합니다.

In [ ]:
label_ref_numbers[20:30]

대부분 1‑3 개의 문서를 참조하고 있으며, 25 번 샘플은 [1, 3] 으로 두 개 문서를 참조했습니다.

In [ ]:
pred_ref_numbers[20:30]

동일한 샘플들에 대한 모델 예측 참조 번호입니다. 25 번 샘플에서 모델은 [1, 2, 3] 으로 과대 예측을 했고, 20 번 샘플에서는 정답이 [1] 인데 [2] 로 예측하여 완전히 틀렸습니다. 테스트 데이터 상위 50 개의 데이터에 대해서 F1 스코어를 계산합니다.

In [ ]:
f1 = calculate_f1_score(label_ref_numbers, pred_ref_numbers)
print(f1)

전체 50 개 샘플에 대한 문서 참조 F1 스코어를 계산합니다. 0.947 로 약 94.7% 의 준수한 성능을 보입니다. 이는 모델이 대부분의 경우에 레이블과 거의 유사하게 올바른 문서를 참조하고 있음을 의미합니다.
